In [1]:
import pandas as pd

classification = pd.read_excel("../data/train_Ametist/classification.xlsx")
train_data = pd.read_excel("../data/train_Ametist/train.xlsx")

In [2]:
def extract_group(text, taget="Группа"):
    text = str(text)
    if taget in text:
        return text.split(":")[-1].replace(f"{taget} ", "").strip()
    return None

In [3]:
classification["group"] = classification["Классификатор строительных ресурсов"].apply(
    lambda x: extract_group(x, "Группа")
)
classification["group"] = classification["group"].fillna(method="ffill")

In [1]:
classification["group"] = classification["Классификатор строительных ресурсов"].apply(
    lambda x: extract_group(x, "Группа")
)
classification["group"] = classification["group"].fillna(method="ffill")

NameError: name 'classification' is not defined

In [5]:
classification["part"] = classification["Классификатор строительных ресурсов"].apply(
    lambda x: extract_group(x, "Раздел")
)
classification["part"] = classification["part"].fillna(method="ffill")

In [6]:
import spacy
import re

nlp = spacy.load(
    "ru_core_news_sm"
)  # нужно скачать: python -m spacy download ru_core_news_sm


def strip_features_and_normaize(text):
    text = re.sub(r"\s*-\s*", "/", text)
    text = re.sub(r"\(.*?\)|\[.*?\]|\{.*?\}", "", text)
    result = nlp(text)
    filtered_result = filter(
        lambda x: x.pos_ in ["NOUN", "ADJ", "VERB", "PROPN"]
        and x.dep_
        in [
            "ROOT",
            "amod",
            "nsubj",
            "advmod",
            "nmod",
            "nsubjpass",
            "nmod:npmod",
            "nmod:poss",
            "nmod:tmod",
            "obl",
        ]
        and len(x.text) > 2,
        result,
    )

    return " ".join(list([e.lemma_.strip() for e in filtered_result]))

In [7]:
classification = classification[4:-1].dropna(subset="Unnamed: 1")
classification = classification[
    classification["Unnamed: 1"].apply(lambda x: "Группа" not in x)
]

In [8]:
classification = classification[
    classification["Классификатор строительных ресурсов"].apply(lambda x: len(x) > 15)
]

In [ ]:
%%time
classification['text'] = classification['Unnamed: 1'].apply(strip_features_and_normaize)

In [115]:
def save_to_fasttext_format(df, filename):
    with open(filename, "w") as f:
        for index, row in df.iterrows():
            f.write(f"__label__{'_'.join(row['group'].split())} {row['Unnamed: 1']}\n")

In [116]:
from sklearn.model_selection import train_test_split


train_df, test_df = train_test_split(classification, test_size=0.2, random_state=42)
save_to_fasttext_format(train_df, "train.txt")
save_to_fasttext_format(test_df, "test.txt")

In [117]:
import fasttext


model = fasttext.train_supervised(
    input="train.txt", dim=50, ws=3, minCount=0, minCountLabel=0, wordNgrams=2
)
results = model.test("test.txt")
results

Read 1M words
Number of words:  53815
Number of labels: 2078
Progress: 100.0% words/sec/thread:  123562 lr:  0.000000 avg.loss:  4.499919 ETA:   0h 0m 0s


(21693, 0.6802194256211681, 0.6802194256211681)

In [18]:
model.predict(
    strip_features_and_normaize(
        "Конструкция габионная из панелей из сетки с размерами ячейки 130х130 мм, из стальной проволоки диаметром 6 мм с цинковым покрытием, со стальным каркасом диаметром 12 мм, в комплекте со стальными U-образными скобами М10 (96 шт), размеры 4000х1000х1000 мм"
    )
)

(('__label__Сетки',), array([0.11886987]))

In [19]:
model.get_sentence_vector(
    strip_features_and_normaize(
        "Конструкция габионная из панелей из сетки с размерами ячейки 130х130 мм, из стальной проволоки диаметром 6 мм с цинковым покрытием, со стальным каркасом диаметром 12 мм, в комплекте со стальными U-образными скобами М10 (96 шт), размеры 4000х1000х1000 мм"
    )
).shape

(100,)

In [118]:
classification["embed"] = classification["text"].apply(
    lambda x: model.get_sentence_vector(x)
)
classification["label"] = classification["group"].apply(
    lambda x: f"__label__{'_'.join(x.split())}"
)

In [119]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


def get_group(text, df):
    text_stripped = strip_features_and_normaize(text)
    text_stripped_embed = model.get_sentence_vector(text_stripped)
    text_group = model.predict(text_stripped)[0][0]

    return text_group


def get_result(text, df):
    text_group = get_group(text, df)
    text_embed = model.get_sentence_vector(text)

    group = df[df["label"] == text_group].reset_index(drop=True)
    group_embed = np.stack(group["embed"].to_list())

    cosine_sims = cosine_similarity(group_embed, text_embed[None, :])
    return group.iloc[np.argmax(cosine_sims)]


sample_idx = 7
result = get_result(train_data.record_name.iloc[sample_idx], classification)

print("Got:", result["Классификатор строительных ресурсов"], result["Unnamed: 1"])
print(
    "Target: ",
    train_data.ref_code.iloc[sample_idx],
    train_data.record_name.iloc[sample_idx],
)

Got: 22.23.19.000.59.1.01.07-0770-000 Блоки дорожные пенополистирольные, модуль упругости при 1 % линейной деформации 2,5 МПа, плотность 15 кг/м3, сопротивление сжатию при 1 % деформации 0,025 МПа
Target:  25.11.23.112.01.7.15.01-1183-000 Анкерный стержень 15,0мм без покрытия 1,00м


In [120]:
train_data["pred"] = train_data["record_name"].apply(lambda x: model.predict(x)[0][0])

In [121]:
def nasral(x):
    try:
        return classification[
            classification["Классификатор строительных ресурсов"] == x
        ]["label"].values[0]
    except:
        return 0

In [122]:
train_data["from_razmetka"] = train_data["ref_code"].apply(lambda x: nasral(x))

In [123]:
(train_data["pred"] == train_data["from_razmetka"]).sum() / len(train_data)

0.0029081844619858743

In [76]:
print(
    classification_report(
        train_data["from_razmetka"].fillna(""), train_data["pred"].fillna("")
    )
)

TypeError: '<' not supported between instances of 'int' and 'str'

In [71]:
train_data

,record_name,record_name_2,ref_code,ref_name,ref_unit,pred,from_razmetka
0,DIN -рейка оцинкованная ТН35-7.5 100 см (Chint),"DIN-рейка 35х7,5 мм длиной 1000 мм",27.33.13.130.20.2.08.01-0001-000,"DIN-рейки металлические, оцинкованные, размеры...",100 шт,__label__Арматура_электромонтажная,__label__DIN-рейки_металлические
1,Анкерный элемент ТехноНиколь 8*45мм,Анкерный элемент TN 8x4.5,25.94.11.190.01.7.15.01-1169-000,"Анкер грунтовый забивной самораскрывающийся, о...",шт,__label__Конструкции_и_детали_конструкций_стро...,__label__Анкеры
2,Анкер забивной М10 DRM 12x40 сталь,Анкер втулочный M10,25.94.11.190.01.7.15.01-0037-000,"Анкер забивной латунный, диаметр внутренней ре...",шт,__label__Анкеры,__label__Анкеры
3,Анкер забивной М10/12x40,Анкер втулочный M10,25.94.11.190.01.7.15.01-0037-000,"Анкер забивной латунный, диаметр внутренней ре...",шт,__label__Материалы_и_изделия_строительного_и_с...,__label__Анкеры
4,Анкер забивной М8 LAZ латунь,Анкер втулочный M8,25.94.11.190.01.7.15.01-0036-000,"Анкер забивной латунный, диаметр внутренней ре...",шт,__label__Материалы_и_изделия_строительного_и_с...,__label__Анкеры
...,...,...,...,...,...,...,...
2402,Корпус замка Фабрика замков L 008,Межкомнатные двери 900х2000,27.12.31.000.20.4.04.04-0002-000,Щит учетно-распределительный встраиваемый ЩУРВ...,шт,__label__Материалы_и_изделия_тепло-_и_звукоизо...,__label__Щиты_учетно-распределительные
2403,Эмаль Кремнийорганическая Термостойкая (8) 81....,Кремнийорганическая эмаль КО-8104,20.30.12.130.14.4.04.04-0010-005,Эмаль кремнийорганическая термостойкая КО-811К...,т,__label__Материалы_лакокрасочные_на_основе_сло...,__label__Эмали_кремнийорганические
2404,Кремнийорганическая эмаль КО-8104,Кремнийорганическая эмаль КО-8104,20.30.12.130.14.4.04.04-0001-000,Эмаль кремнийорганическая термостойкая КО-88,т,__label__Материалы_лакокрасочные_на_основе_сло...,__label__Эмали_кремнийорганические
2405,Кремнийорганическая эмаль КО-8104 (серебристо ...,Кремнийорганическая эмаль КО-8104,20.30.12.130.14.4.04.04-0001-000,Эмаль кремнийорганическая термостойкая КО-88,т,__label__Материалы_лакокрасочные_на_основе_сло...,__label__Эмали_кремнийорганические
